### keras-yolo3 패키지를 이용하여 Yolo와 tiny Yolo 기반으로 이미지와 영상 Object Detection 수행
* 다크넷에서 Pretrained된 yolo/tiny-yolo weights 모델을 다운로드
* 다운로드한 다크넷 weight 파일을 기반으로 keras-yolo3에서 사용할 수 있는 weight 파일로 변환 후 이를 이용하여 Object Detection 수행

### 본 실습 예제는 GPU를 활용하므로 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택해 주십시요.

### tensorflow, keras 설치 및 강의 실습코드/데이터 Download
#### 공지

현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다.

Colab 버전colab에서 pip 명령어를 이용하여 tensorflow 1.15, keras 2.3를 소스코드 커널 기동시 마다 설치해야 합니다

Colab의 tensorflow는 2020년 기준으로 2.2 이며, keras는 2.3입니다. 실습코드는 tensorflow 1.13과 1.15, keras 2.2와 2.3 기준으로 되어 있으므로 이를 downgrade해야 합니다.

pip를 이용하여 tensorflow 1.15을 설치하면 자동으로 downgrade 됩니다.

OpenCV는 Colab에서 이미 설치 되어 있으니 추가설치는 필요 없습니다.

강의 실습코드와 데이터는 https://github.com/chulminkw/DLCV.git 에서 다운로드 할 수 있습니다. 

In [1]:
# 현재 디렉토리는 /content이며 이 디렉토리를 기준으로 실습코드와 데이터를 다운로드 합니다. 
!pwd
!rm -rf DLCV
!git clone https://github.com/chulminkw/DLCV.git
# DLCV 디렉토리가 Download되고 DLCV 밑에 Detection과 Segmentation 디렉토리가 있는 것을 확인
!ls -lia 
!ls -lia DLCV

/content
Cloning into 'DLCV'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 185 (delta 10), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (185/185), 134.41 MiB | 13.41 MiB/s, done.
Resolving deltas: 100% (67/67), done.
total 20
3932195 drwxr-xr-x 1 root root 4096 Aug 17 10:01 .
3946108 drwxr-xr-x 1 root root 4096 Aug 17 09:48 ..
3932196 drwxr-xr-x 1 root root 4096 Aug 10 21:25 .config
3946196 drwxr-xr-x 7 root root 4096 Aug 17 10:01 DLCV
3407899 drwxr-xr-x 1 root root 4096 Jul 30 16:30 sample_data
total 6456
3946196 drwxr-xr-x 7 root root    4096 Aug 17 10:01 .
3932195 drwxr-xr-x 1 root root    4096 Aug 17 10:01 ..
3946273 drwxr-xr-x 2 root root    4096 Aug 17 10:01 colab_tf115_modify_files
3946275 drwxr-xr-x 6 root root    4096 Aug 17 10:01 data
3946239 drwxr-xr-x 8 root root    4096 Aug 17 10:01 Detection
3946224 -rw-r--r-- 1 root root 6577714 Aug 17 10:01 DLCV_Colab_

In [2]:

#현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 
#때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다.
# tensorflow 1.15을 설치합니다. 자동으로 tensorflow 2.2가 1.15로 downgrade 됩니다. 
!pip install tensorflow-gpu==1.15.2 
# keras 2.3를 설치합니다. 
!pip install keras==2.3.0

     |████████████████████████████████| 411.0MB 43kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 42.1MB/s 
     |████████████████████████████████| 512kB 39.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=72f4e4b7cb68aae47f11c5e7c531b722137784d2f759b32d83e87c71809d2fdd
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [3]:
# tensorflow는 1.15, keras는 2.3 버전 확인
# GPU가 세팅되어 있지 않으면 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택한 후 런타임 다시 시작을 선택하고 처음 부터인 tensorflow, keras 설치 부터 다시 시작. 
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

# gpu가 세팅드어 있는지 확인. 
tf.test.gpu_device_name()

Using TensorFlow backend.


1.15.2
2.3.0


'/device:GPU:0'

In [ ]:
import os
import sys
import random
import math
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

### 코랩 버전은 아래를 이용하여 keras-yolo3 패키지를 download하여 /content/DLCV/Detection/yolo 밑에 설치 

In [ ]:
%cd /content/DLCV/Detection/yolo
!git clone https://github.com/qqwweee/keras-yolo3.git
!ls -lia /content/DLCV/Detection/yolo/keras-yolo3

#### Local Directory 상에서 yolo package를 import 함. 
* keras-yolo3는 setup을 제공하지 않으므로 local directory상에서 바로 package를 import함. 
* 이를 위해 keras-yolo3를 system path에 추가  
* keras-yolo3 디렉토리에 있는 yolo.py에서 YOLO class를 import하여 사용. 

In [ ]:
# 코랩 버전은 아래와 같이 절대 경로를 지정하여 Local Package 지정. 
default_dir = '/content/DLCV'
default_yolo_dir = os.path.join(default_dir, 'Detection/yolo')

LOCAL_PACKAGE_DIR = os.path.abspath(os.path.join(default_yolo_dir,'keras-yolo3'))
print(LOCAL_PACKAGE_DIR)
sys.path.append(LOCAL_PACKAGE_DIR)

from yolo import YOLO

In [ ]:
# YOLO 클래스는 model_path, achors_path, classes_path를 model_data 밑에 파일로 가짐. 변경을 위해서는 yolo.py 파일에서 YOLO 클래스코드를  직접 변경 필요. 

print(LOCAL_PACKAGE_DIR)
!ls /home/chulmin.kwon45/DLCV/Detection/yolo/keras-yolo3
!cat /home/chulmin.kwon45/DLCV/Detection/yolo/keras-yolo3/yolo.py

#### YOLO 객체를 사용하기 위한 모델 파일 설정 및 소스 코드 변경
* 다크넷에서 Yolo V3 Weight 모델 파일을 다운로드 받은 뒤 이를 keras-yolo3용으로 모델 파일 변경
* model_data 디렉토리 밑에 yolo_anchors.txt, coco_classes.txt 가 있는지 확인.


In [ ]:
!ls /content/DLCV/Detection/yolo/keras-yolo3

In [ ]:
# keras-yolo3 디렉토리 밑에서 아래 명령어로 다크넷에서 Yolov3 weight를 다운로드 받고 이를 keras-yolo3용으로 모델 파일 변경
#!wget https://pjreddie.com/media/files/yolov3.weights
#!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5
# convert.py를 수행하면 yolo anchor값이 yolo_anchors.txt 파일이 자동으로 생성됨. 
# coco label과 클래스 매핑은 0부터 매핑함.(0 => person)

In [ ]:
# 코랩 버전은 아래를 이용하여 yolov3.weights 파일을 download 받고, convert.py 를 수행하여 model_data 밑에 yolo.h5 파일 생성 수행. 
%cd /content/DLCV/Detection/yolo/keras-yolo3 
# yolo 공식 사이트에서 download시 download 속도가 약 25분 정도 소요됨. 강의 github에서 다운로드 요망. 
#!wget  https://pjreddie.com/media/files/yolov3.weights
# 강의 실습 github에서 다운로드
!wget https://github.com/chulminkw/DLCV/releases/download/1.0/yolov3.weights

# yolov3.weights를 keras-yolo3에서 사용할 수 있도록 yolo.h5 로 변환
!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5
# model_data 밑에 yolo.h5 파일이 생성되었는지 확인. 
!ls /content/DLCV/Detection/yolo/keras-yolo3/model_data

In [ ]:
import sys
import argparse
from yolo import YOLO, detect_video
#keras-yolo에서 image처리를 주요 PIL로 수행. 
from PIL import Image

# YOLO 객체 생성. config는 default로 keras-yolo3 디렉토리에 있는 yolov3.cfg를 적용. 
# 코랩 버전은 절대 경로로 각 생성 인자값 입력
default_yolo_dir = '/content/DLCV/Detection/yolo'
config_dict = {}
yolo = YOLO(model_path=os.path.join(default_yolo_dir, 'keras-yolo3/model_data/yolo.h5'),
            anchors_path=os.path.join(default_yolo_dir, 'keras-yolo3/model_data/yolo_anchors.txt'),
            classes_path=os.path.join(default_yolo_dir, 'keras-yolo3/model_data/coco_classes.txt'))

#### 단일 이미지 Object Detection 

In [ ]:
# 원본 이미지 보기 
default_dir = '/content/DLCV'
img = Image.open(os.path.join(default_dir, 'data/image/beatles01.jpg'))

plt.figure(figsize=(12, 12))
plt.imshow(img)

In [ ]:
# yolo.detect_image() 메소드는 PIL package를 이용하여 image 작업 수행. keras-yolo3/font 디렉토리를 상위 디렉토리로 복사 해야함.  
%cd /content/DLCV/Detection/yolo
!cp -rf keras-yolo3/font ./font

In [ ]:
img = Image.open(os.path.join(default_dir, 'data/image/beatles01.jpg'))
detected_img = yolo.detect_image(img)

plt.figure(figsize=(12, 12))
plt.imshow(detected_img)

### Video Object Detection 수행

In [ ]:
import cv2
import time

def detect_video_yolo(model, input_path, output_path=""):
    
    start = time.time()
    cap = cv2.VideoCapture(input_path)
    
    #codec = cv2.VideoWriter_fourcc(*'DIVX')
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_fps = cap.get(cv2.CAP_PROP_FPS)
    vid_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)
    
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt, '원본 영상 FPS:',vid_fps)
    index = 0
    while True:
        hasFrame, image_frame = cap.read()
        if not hasFrame:
            print('프레임이 없거나 종료 되었습니다.')
            break
        start = time.time()
        # PIL Package를 내부에서 사용하므로 cv2에서 읽은 image_frame array를 다시 PIL의 Image형태로 변환해야 함.  
        image = Image.fromarray(image_frame)
        # 아래는 인자로 입력된 yolo객체의 detect_image()로 변환한다.
        detected_image = model.detect_image(image)
        # cv2의 video writer로 출력하기 위해 다시 PIL의 Image형태를 array형태로 변환 
        result = np.asarray(detected_image)
        index +=1
        print('#### frame:{0} 이미지 처리시간:{1}'.format(index, round(time.time()-start,3)))
        
        vid_writer.write(result)
    
    vid_writer.release()
    cap.release()
    print('### Video Detect 총 수행시간:', round(time.time()-start, 5))

In [ ]:
default_dir = '/content/DLCV'
detect_video_yolo(yolo, os.path.join(default_dir, 'data/video/Night_Day_Chase.mp4'),
                  os.path.join(default_dir, 'data/output/Night_Day_Chase_yolo_01.avi'))

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 이를 위해 google drive를 colab에 mount 수행. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')


In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 
## My Drive 디렉토리 이름에 공란이 있으므로 ' '로 묶습니다. 
!cp /content/DLCV/data/output/Night_Day_Chase_yolo_01.avi '/content/gdrive/My Drive/Night_Day_Chase_yolo_01.avi'

### tiny Yolo를 이용하여 이미지와 영상 object detection 수행. 
* tiny yolo weights파일은 https://pjreddie.com/media/files/yolov3-tiny.weights 에서 다운로드 받을 수 있음. 
* 다운로드 받은 tiny-yolo weight파일을 keras-yolo3에서 사용할 수 있게 Convert 수행 후 YOLO객체에서 로딩하여 사용 

In [ ]:
# wget https://pjreddie.com/media/files/yolov3-tiny.weights
#!python convert.py yolov3-tiny.cfg ./model_data/yolov3-tiny.weights model_data/yolo-tiny.h5

In [ ]:
# 코랩 버전은 아래를 이용하여 yolov3-tiny.weights 파일을 download 받고, convert.py 를 수행하여 model_data 밑에 yolo-tiny.h5 파일 생성 수힝. 
%cd /content/DLCV/Detection/yolo/keras-yolo3 
!wget https://pjreddie.com/media/files/yolov3-tiny.weights
!python convert.py yolov3-tiny.cfg yolov3-tiny.weights model_data/yolo-tiny.h5
# model_data 밑에 yolo-tiny.h5 파일이 생성되었는지 확인. 
!ls /content/DLCV/Detection/yolo/keras-yolo3/model_data

#### tiny yolo weight 파일과 anchor 파일, coco 클래스 파일을 YOLO 객체 생성 시 인자로 입력

In [ ]:
default_yolo_dir = '/content/DLCV/Detection/yolo'
config_dict = {}

tiny_yolo = YOLO(model_path=os.path.join(default_yolo_dir,'keras-yolo3/model_data/yolo-tiny.h5'),
            anchors_path=os.path.join(default_yolo_dir,'keras-yolo3/model_data/tiny_yolo_anchors.txt'),
            classes_path=os.path.join(default_yolo_dir,'keras-yolo3/model_data/coco_classes.txt'))

#### 단일 이미지 Object Detection

In [ ]:
default_dir = '/content/DLCV'
img = Image.open(os.path.join(default_dir, 'data/image/beatles01.jpg'))
detected_img = tiny_yolo.detect_image(img)

plt.figure(figsize=(12, 12))
plt.imshow(detected_img)

#### Video Object Detection

In [ ]:
default_dir = '/content/DLCV'
detect_video_yolo(tiny_yolo, os.path.join(default_dir, 'data/video/Night_Day_Chase.mp4'), os.path.join(default_dir, 'data/output/Night_Day_Chase_tiny_yolo01.avi'))

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 이를 위해 google drive를 colab에 mount 수행. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 
## My Drive 디렉토리 이름에 공란이 있으므로 ' '로 묶습니다. 
!cp /content/DLCV/data/output/Night_Day_Chase_tiny_yolo01.avi '/content/gdrive/My Drive/Night_Day_Chase_tiny_yolo01.avi'